In [2]:
import os
import requests
import pandas as pd
import random
from typing import Dict, List
from sqlalchemy import create_engine
from dotenv import load_dotenv
import re
from utils import fetch_last_news
from multiprocessing import Pool
from tqdm.autonotebook import tqdm

/tmp/ipykernel_76104/3318184949.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
load_dotenv()
PG_DSN = (
    f"postgresql://{os.getenv('PG_USER')}:"
    f"{os.getenv('PG_PASS')}@"
    f"{os.getenv('PG_HOST')}:"
    f"{os.getenv('PG_PORT')}/"
    f"{os.getenv('PG_DB')}"
)
OPENROUTER_KEY = os.getenv('OPENROUTER_KEY')
engine = create_engine(PG_DSN)

In [4]:
EXPECTED_KEYS = {"easy", "medium", "hard"}

def parse_gpt_output(raw: str, *, article_id: str = "") -> Dict[str, str]:
    """
    Преобразует сырой текст от gpt.generate_for_article
    в словарь {"easy": "...", "medium": "...", "hard": "..."}.
    • Игнорирует пустые строки и строки без “:” или “-”/“–”/“—”.
    • Ключи чувствительны к регистру, приводятся к нижнему.
    • Если ключ отсутствует, возвращается пустая строка.
    """
    out = {k: "" for k in EXPECTED_KEYS}

    for line in raw.splitlines():
        if not line.strip():
            continue

        # Попробуем найти “ключ : значение” или “ключ - значение” и т.д.
        m = re.match(r"^\s*([\w]+)\s*[:\-–—]\s*(.+)$", line)
        if not m:
            # при желании можно раскомментировать для диагностики
            # print(f"[Warn] article_id={article_id} → could not parse line: {line!r}")
            continue

        k, v = m.group(1).lower(), m.group(2).strip()
        if k in EXPECTED_KEYS:
            out[k] = v

    return out


In [5]:
class GPTSynthesizer:
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.url = "https://openrouter.ai/api/v1/chat/completions"

    def _gpt_query(self, system_prompt: str, prompt: str) -> str:
        messages: List[Dict[str, str]] = []
        if system_prompt:
            messages.append({
                "role": "system",
                "content": "Всегда отвечай на русском языке. " + system_prompt
            })
        messages.append({"role": "user", "content": prompt})

        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json",
        }

        payload = {
            "model": "qwen/qwen-turbo",
            "messages": messages,
            "temperature": 0,
            "max_tokens": 512,
            "seed": 42,
        }

        response = requests.post(self.url, headers=headers, json=payload)
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]

    def generate_for_article(self, title: str, anons: str, body: str, *, article_id: str = "") -> Dict[str, str]:
        """
        Для переданной новости (title, anons, body) формирует 3 поисковых запроса
        с разными уровнями сложности: лёгкий, средний, сложный.
        Возвращает словарь {"easy": ..., "medium": ..., "hard": ...}.
        """
        snippet = body[:200].replace("\n", " ")
        full_context = f"Заголовок: {title}\nАнонс: {anons or ''}\nТело (фрагмент): {snippet}..."
        system_prompt = (
            "Ты генерируешь три реалистичных поисковых запроса человека в поисковике разного уровня сложности, "
            "чтобы найти именно эту новость на новостном портале. "
            "Первый запрос — максимально буквальный, в точности, как в новости"
            "Второй — чуть более обобщённый (с синонимами). Как бы мог сформулировать эту новость человек"
            "Третий — максимально абстрактный, где используются контекстные формулировки и косвенные признаки, "
            "возможны реалестичные опечатки, чтобы тестировать поисковую систему по новостям. Главное, чтобы запросы были, как от реального человека"
        )
        prompt = (
            f"{full_context}\n\n"
            "Сгенерируй три отдельных строки запроса человека к новостному порталу, помеченные как:\n"
            "easy: \n"
            "medium: \n"
            "hard: "
        )

        raw = self._gpt_query(system_prompt, prompt)

        parsed = parse_gpt_output(raw, article_id=article_id)
        return parsed

In [10]:
df_sample = fetch_last_news(limit=5)
gpt = GPTSynthesizer(api_key=OPENROUTER_KEY)


In [11]:
def _process_row_tuple(args):
    art_id, title, anons, body = args
    try:
        gens = gpt.generate_for_article(title, anons, body, article_id=art_id)
        # Если нужны дополнительные проверки:
        # if not all(k in gens for k in ("easy", "medium", "hard")):
        #     raise ValueError(f"Missing keys in gens for article {art_id}: {gens.keys()}")
    except Exception as e:
        print(f"[Error] article_id={art_id} → {type(e).__name__}: {e}")
        gens = {"easy": "", "medium": "", "hard": ""}
    return art_id, gens

# -----------------------------
# 5) Основная функция для генерации в несколько процессов
# -----------------------------
def generate_synthetic_mp(df, num_workers=None, chunksize=10):
    """
    Обрабатывает каждую строку df параллельно, используя multiprocessing.Pool.
    Возвращает список кортежей: (article_id, difficulty, generated_text).
    """
    arg_list = [
        (row["id"], row["title"], row["anons"], row["body"])
        for _, row in df.iterrows()
    ]

    synthetic = []
    with Pool(processes=num_workers) as pool:
        for art_id, gens in tqdm(
            pool.imap_unordered(_process_row_tuple, arg_list, chunksize=chunksize),
            total=len(arg_list),
            desc="Generating synthetic queries",
        ):
            synthetic.append((art_id, "easy",   gens.get("easy", "")))
            synthetic.append((art_id, "medium", gens.get("medium", "")))
            synthetic.append((art_id, "hard",   gens.get("hard", "")))

    return synthetic

In [ ]:
synthetic_queries = generate_synthetic_mp(df_sample, num_workers=None, chunksize=5)

Generating synthetic queries:   0%|          | 0/5 [00:00<?, ?it/s]

Generating synthetic queries: 100%|██████████| 5/5 [00:08<00:00,  1.74s/it]


In [ ]:
synthetic_queries

[('021f3d7d0c656ce8c3df769e6bf8f640',
  'easy',
  'Пожар на Стромынке тушили около двух часов'),
 ('021f3d7d0c656ce8c3df769e6bf8f640',
  'medium',
  'Крупный пожар на востоке Москвы на Стромынке, здание старой постройки, обрушение перекрытий'),
 ('021f3d7d0c656ce8c3df769e6bf8f640',
  'hard',
  'восток москвы сормынка пожар здание старое обрушение перекрытий суббота вечером'),
 ('2897c30962778a42f38048ec7ba79b1e',
  'easy',
  'Терпению – конец: Европа протестует против мигрантов'),
 ('2897c30962778a42f38048ec7ba79b1e',
  'medium',
  'Европа бунтует из-за мигрантов, протесты проходят в разных городах'),
 ('2897c30962778a42f38048ec7ba79b1e',
  'hard',
  'Европские протесты против имграции, беспорядки в городах, рукоприкладство'),
 ('7cc2961142a9a818763e4c88d89545e4',
  'easy',
  'Пропавший на Эльбрусе лыжник из Москвы найден спасателями'),
 ('7cc2961142a9a818763e4c88d89545e4',
  'medium',
  'Лыжник из Москвы, пропавший на Эльбрусе, найден и жив'),
 ('7cc2961142a9a818763e4c88d89545e4',
  '

In [ ]:
df_synth = pd.DataFrame(synthetic_queries, columns=["article_id", "difficulty", "query"])
queries_gt: Dict[str, List[str]] = {}
for art_id, difficulty, q in tqdm(synthetic_queries):
    queries_gt[q] = [art_id]

100%|██████████| 300/300 [00:00<00:00, 1459734.57it/s]


In [ ]:
df_synth.to_csv("synthetic_queries_first.csv", index=False)